# Shots Ranking
Creazione di un ranking per gli shots in base ai tag in essi contenuti. 
Se uno shot ha un tag più frequente ha un livello di rank più alto

In [ ]:
import pymongo
from pymongo import MongoClient

from pandas import DataFrame, read_csv
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

client = MongoClient('mongodb://127.0.0.1:27017')
db = client.sample

cu = db.users.find()
cs = db.shots.find()
ct = db.teams.find()
cutfs = db.user_team_first_shot.find()
cr = db.shots_ranking.find()

In [ ]:
usernames_no_repeated = []
for user in cutfs:
    if user['user'] not in usernames_no_repeated:
        usernames_no_repeated.append(user['user'])

In [ ]:
tags_found = []

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/features/shots_tags_frequences.txt", "r") as ins:
    for line in ins:
        tag = line[0:line.rfind('-')-1]
        count = line[line.rfind('-')+2:len(line)-1]
        elem = {'tag': tag, 'count': int(count)}
        tags_found.append(elem)

In [ ]:
def n_most_frequent(n, tag_list):
    
    tags = []
    
    max_tag_prev = {'count':1000000}
    for i in range(0,n):
        max = 0
        max_tag = {}
        for t in tag_list:
            if int(t['count']) > max and int(t['count']) < int(max_tag_prev['count']):
                max = int(t['count'])
                max_tag = t
                
        
        max_tag_prev = max_tag
        tags.append(max_tag)
    return tags

In [ ]:
tags_str = []
for tag in n_most_frequent(10, tags_found):
    tags_str.append(tag['tag'])


In [ ]:
%%time

ranking = []

for u in usernames_no_repeated:
    active_user = db.shots.find({'username': u})
    if active_user.count() > 0: 
        for s in active_user[0]['shots']:
            shot_rank = 0
            tags = s['tags']
            for t in tags:
                if t in tags_str:
                    shot_rank = shot_rank + 1
                    
            elem = {'shot': s['id'], 'rank': shot_rank}
            ranking.append(elem)

In [ ]:
for r in ranking:
    db.shots_ranking.insert_one(r)

### Funzione che restituisce il rank di uno shot

In [ ]:
def shot_rank(shot_id):
    return db.shots_ranking.find({'shot': shot_id})[0]['rank']

In [ ]:
shot_rank(2686725)